# Splash dynamics on a cone surface



## Documentation and Imports


Created on 27-07-21

Author: Valentin Laplaud

Formulas and graphs describing the dynamics of a drop splashing on a cone

In [1]:
# plotting stuff
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline

COLOR = 'white'
COLOR2 = 'black'

mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR

mpl.rcParams["figure.facecolor"] = COLOR2
mpl.rcParams["axes.facecolor"] = COLOR2
mpl.rcParams["savefig.facecolor"] = COLOR2

import matplotlib.pyplot as plt
from cycler import cycler


# numbers handling
import numpy as np
from numpy import pi
import pandas as pd

# signal processing 
from scipy.signal import savgol_filter, correlate, correlation_lags
from scipy.interpolate import interp1d


# to hide known warnings
import warnings
warnings.filterwarnings("ignore")

# General system functions
import os
import shutil
import sys

import time

# my functions
sys.path.append('../PythonFuncs/')
import VallapFunc_DP as vf
import DropGeometryClasses as dgc
import DropGeometryFuncs as dgf


## Drop impact characteristics

### Top view schematics

In [2]:
# Xc = 0
# Yc = 0
# Rc = 5

# Xd = 6
# Yd = 0
# Rd = 4

# r = vf.dist(Xd,Yd,Xc,Yc)


# tx = np.linspace(0,2*pi,100)

# fig,ax = plt.subplots(dpi=600)
# ax.set_aspect('equal', adjustable='box') 
# ax.axis('off')

# # Cone
# ax.plot(Xc,Yc,'ow',ms = 2)
# ax.plot(Xc + Rc*np.cos(tx),Yc + Rc*np.sin(tx), '-w')
# ax.plot([Xc, Xc + Rc*np.cos(tx[30])], [Yc, Yc + Rc*np.sin(tx[30])], '--w', lw = 0.5)
# ax.text(Xc + Rc*np.cos(tx[36])/2,Yc+ Rc*np.sin(tx[36])/2, 'Rc', c = 'w', size = Rc*2)


# # Drop
# ax.plot(Xd,Yd,'ob',ms = 2)
# ax.plot(Xd + Rd*np.cos(tx),Yd + Rd*np.sin(tx), '-b')
# ax.plot([Xd, Xd + Rd*np.cos(tx[90])], [Yd, Yd + Rd*np.sin(tx[90])], '--b', lw = 0.5)
# ax.text(Xd + Rd*np.cos(tx[93])/2,Yd + Rd*np.sin(tx[93])/2, 'Rd', c = 'b', size = Rd*2)

# # Geometry definition
# ax.plot([Xc, Xd], [Yc, Yd], '--r', lw = 0.5, zorder = -1)
# ax.text((Xc+Xd)/2,(Yc+Yd)/2+Rd/10,'r', c= 'r', size = 10);


### Adimensional numbers

In [3]:
# ### Parameters
# V = 4 # [m/s] drop impact speed
# L = 0.003 # [m] cone diameter
# rho = 1000 # [kg/m3] water density
# mu = 0.001 # [Pa.s] water viscosity
# gamma = 0.07 # [N/m] water surface tension
# g = 9.8 # [m/s²] gravity constant

# ### Adimensional numbers

# # Reynolds (viscosity vs. intertia)
# Re = rho*V*L/mu
# print('Reynolds = ' + str(Re))

# # Weber (capilarity vs. inertia)
# We = rho*L*V**2/gamma
# print('Weber = ' + str(We))

# # Froude (gravity vs. intertia)
# Fr = V/np.sqrt(g*L)
# print('Froude = ' + str(Fr))

### Plots of fall probability


In [4]:

# dropR = 3
# coneR = 4 # Cone radius
# Alpha = pi/4 # Cone vertical angle

# xcircle = coneR*np.cos(np.linspace(0,2*pi,100))
# ycircle = coneR*np.sin(np.linspace(0,2*pi,100))
# xgrid = np.arange(-(coneR+dropR), coneR+dropR, 0.1)
# ygrid = np.arange(-(coneR+dropR), coneR+dropR, 0.1)
# xx, yy = np.meshgrid(xgrid, ygrid)
# mask = np.sqrt(xx**2+yy**2)<(coneR+dropR)
# z = dgf.ProbaFall(np.sqrt(xx[mask]**2+yy[mask]**2),dropR,coneR)


# ### Drop-Cone distance
# rstep = (coneR+dropR)/98
# nstep = int(np.ceil((coneR+dropR)/rstep))
# r = np.linspace(rstep/2,coneR+dropR-3*rstep/2,nstep) # impact position
# Pr = dgf.ProbaFall(r,dropR,coneR)


# # probability of falling at a certain distance of the cone center
# fig10,ax10 = plt.subplots(dpi = 500)
# fig10.suptitle("Probability of falling at a distance 'r' from the center \n dropR = " + str(dropR) + " mm, coneR = " + str(coneR) + " mm. ")
# ax10.plot(r/(coneR+dropR)*100,Pr,'o', ms = 1)
# ax10.set_xlabel(' r  [% of (coneR+dropR)]');
# ax10.set_ylabel('Probability [%]');
# fig10.tight_layout()

# fig11, ax11 = plt.subplots(dpi = 500)
# ax11.set_aspect('equal', adjustable='box')
# ax11.set_xlabel('X')
# ax11.set_ylabel('Y')
# fig11.suptitle("Probability of falling at a distance 'r' from the center \n dropR = " + str(dropR) + " mm, coneR = " + str(coneR) + " mm. ")


# ax11.plot(xcircle,ycircle,'-w')
# sc11 = ax11.scatter(xx[mask],yy[mask],c=z,s=1,cmap='jet')
# fig11.colorbar(sc11, ax = ax11, label = 'proba (%)',shrink = 0.6)


## Plots

In [5]:
savepath = r'D:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\Model\Images'

#### Simulations

In [6]:

# fig2,ax2 = plt.subplots(dpi = 500)
# fig2.suptitle("Fraction of drop volume impacting the cone\n dropR = " + str(dropR) + " mm, coneR = " + str(coneR) + " mm. ")
# ax2.plot(r/(coneR+dropR)*100,volFrac(r,dropR,coneR),'o', ms = 1)
# ax2.set_xlabel(' r  [% of (coneR+dropR)]');
# ax2.set_ylabel('Volume fraction [%]');

# JVFrac = JetVolFrac2D(coneR,Alpha,dropR,r,300)
# JVFrac3D = JetVolFrac3D(coneR,Alpha,dropR,r,300)
# DropVolFrac = volFrac(r,dropR,coneR)

# fig3,ax3 = plt.subplots(dpi = 500)
# fig3.suptitle("Fraction of impacting volume forming a jet\n dropR = " + str(dropR) + " mm, coneR = " + str(coneR) + " mm. ")
# ax3.plot(r/(coneR+dropR)*100,JVFrac,'ro',ms = 1,label='Simulation');
# #ax3.plot(r,JVFrac3D,'mo',ms = 1,label='Simulation 3D');
# ax3.plot(r[r>coneR]/(coneR+dropR)*100,splashFracJet(r[r>coneR],coneR,dropR,Alpha),'--',lw=1.5,label='Analytical solution');
# ax3.legend();
# ax3.set_xlabel(' r  [% of (coneR+dropR)]');
# ax3.set_ylabel('Volume fraction [%]');

# fig31,ax31 = plt.subplots(dpi = 500)
# fig31.suptitle("Fraction of impacting volume forming a jet\n dropR = " + str(dropR) + " mm, coneR = " + str(coneR) + " mm. ")
# ax31.plot(r/(coneR+dropR)*100,JVFrac,'ro',ms = 1,label='Simulation 2D');
# ax31.plot(r/(coneR+dropR)*100,JVFrac3D,'mo',ms = 1,label='Simulation 3D');
# ax31.plot(r[r>coneR]/(coneR+dropR)*100,splashFracJet(r[r>coneR],coneR,dropR,Alpha),'--',lw=1.5,label='Analytical solution');
# ax31.legend();
# ax31.set_xlabel(' r  [% of (coneR+dropR)]');
# ax31.set_ylabel('Volume fraction [%]');


# fig4,ax4 = plt.subplots(dpi = 500)
# fig4.suptitle("Drop volume fraction in the jet\n dropR = " + str(dropR) + " mm, coneR = " + str(coneR) + " mm. ")
# ax4.plot(r/(coneR+dropR)*100,volFrac(r,dropR,coneR)*JVFrac3D/100,'o',ms = 1)
# ax4.set_xlabel(' r  [% of (coneR+dropR)]');
# ax4.set_ylabel('Drop volume fraction in the jet [%]');


# fig41,ax41 = plt.subplots(dpi = 500)
# fig41.suptitle("Proba * Drop volume fraction in the jet\n dropR = " + str(dropR) + " mm, coneR = " + str(coneR) + " mm. ")
# ax41.plot(r/(coneR+dropR)*100,volFrac(r,dropR,coneR)*JVFrac3D/100*Pr,'o',ms = 1)
# ax41.set_xlabel(' r  [% of (coneR+dropR)]');
# ax41.set_ylabel('Proba * Drop volume fraction in the jet [%]');


##### Drop size variations

In [7]:
# Alpha = pi/4 # Cone vertical angle

# coneR = 3 # cone radius 
# dropRs = np.round(np.multiply([1/5, 1/4, 1/3, 1/2, 1, 2, 3, 4, 5],coneR)*100)/100 # Drop radii

# coneR = 1 # cone radius 
# dropRs = np.round(np.multiply([0.5, 0.75, 1, 1.25, 1.5, 1.75, 2],coneR)*100)/100 # Drop radii


# fig2,ax2 = plt.subplots(dpi = 500)
# fig2.suptitle("Fraction of drop volume impacting the cone\n coneR = " + str(coneR) + " mm")
# ax2.set_xlabel(' r  [% of (coneR+dropR)]');
# ax2.set_ylabel('Volume fraction [%]');

# fig3,ax3 = plt.subplots(dpi = 500)
# fig3.suptitle("Fraction of impacting volume forming a jet\n coneR = " + str(coneR) + " mm")
# ax3.set_xlabel(' r  [% of (coneR+dropR)]');
# ax3.set_ylabel('Volume fraction [%]');

# fig4,ax4 = plt.subplots(dpi = 500)
# fig4.suptitle("Drop volume fraction in the jet\n coneR = " + str(coneR) + " mm")
# ax4.set_xlabel(' r  [% of (coneR+dropR)]');
# ax4.set_ylabel('Drop volume fraction in the jet [%]');

# fig5,ax5 = plt.subplots(dpi = 500)
# fig5.suptitle("Real volume in the jet\n coneR = " + str(coneR) + " mm")
# ax5.set_xlabel(' r  [% of (coneR+dropR)]');
# ax5.set_ylabel('Volume in the jet [mm**3]');
    

# for dropR in dropRs:

#     ### Drop-Cone distance   
#     rstep = (coneR+dropR)/98
#     nstep = int(np.ceil((coneR+dropR)/rstep))
#     r = np.linspace(rstep/2,coneR+dropR-3*rstep/2,nstep) # impact position
        
#     rRef = r/(coneR+dropR)*100
    
#     # simu
#     JVFrac = JetVolFrac3D(coneR,Alpha,dropR,r,1000)
#     DropVolFrac = volFrac(r,dropR,coneR)
    
#     # plot
#     ax2.plot(rRef,DropVolFrac,'o-', ms = 1,lw=1,label = 'dropR = ' + str(dropR) + ' mm')

#     ax3.plot(rRef,JVFrac,'o-',ms = 1,lw=1, label = 'dropR = ' + str(dropR) + ' mm');

#     ax4.plot(rRef,DropVolFrac*JVFrac/100,'o-',ms = 1,lw=1, label = 'dropR = ' + str(dropR) + ' mm')

#     ax5.plot(rRef,DropVolFrac*JVFrac/10000*dropR**3*4/3*np.pi,'o-',ms = 1,lw=1, label = 'dropR = ' + str(dropR) + ' mm')


# ax2.legend(fontsize = 'xx-small');
# ax3.legend(fontsize = 'xx-small');
# ax4.legend(fontsize = 'xx-small');
# ax5.legend(fontsize = 'xx-small');
# plt.show()


## Test Zone

### Classes 

#### Tests and illustrations

In [8]:
## Definig diverse drops and cones

D0 = dgc.Drop(2.9,0.2,30,4)
D1 = dgc.Drop(2.9,1.5,30,4)
D2 = dgc.Drop(2.9,2.5,30,4)
D3 = dgc.Drop(2.9,3.5,30,4)
D4 = dgc.Drop(2.9,4.5,30,4)
D5 = dgc.Drop(2.9,5.5,30,4)
D6 = dgc.Drop(2.9,6.5,30,4)
D7 = dgc.Drop(2.9,7.5,30,4)

P = dgc.Cone(2.7,np.pi/2) # flat pillar

# Experimental cones
C45_27 = dgc.Cone(2.7,np.pi/4) 
C45_35 = dgc.Cone(3.5,np.pi/4) 
C45_54 = dgc.Cone(5.4,np.pi/4) 

C60 = dgc.Cone(5,np.pi/3)
C30 = dgc.Cone(5,np.pi/6)



In [9]:
## Computing impacts

# I_D0_C45 = C45_35.impact(D0)
# I_D1_C45 = C45_27.impact(D1)

dgc.Cone(3,np.pi/6) .impact(dgc.Drop(2.9,3.5,50,4)).plot_3Dview()

# # I_D4_P = P.impact(D4)
# # I_D1_C30 = C30.impact(D1)
# # I_D1_C60 = C60.impact(D1)


# I_D2_C45 = C45_27.impact(D2)
# I_D3_C45 = C45_27.impact(D3)
# I_D4_C45 = C45_27.impact(D4)
# I_D5_C45 = C45_27.impact(D5)
# I_D6_C45 = C45_27.impact(D6)
# I_D7_C45 = C45_27.impact(D7)


In [10]:
# # Plotting stuff
# # C45_27.draw(nolabels=True,dropview = 'impact',drop=D3,title='45° cone + D1')
# # I_D1_C45.plot_JetFrac(title='Drop fraction in the jet, cone of 45°',nolabels=True)


# I_D1_C45.plot_splash_traj(np.linspace(0,3.5,50),nolabels=True,title='Cone of 45°')

# I_D2_C45.plot_splash_traj(np.linspace(0,3,50),nolabels=True,title='Cone of 45°')

# I_D3_C45.plot_splash_traj(np.linspace(0,2.5,50),nolabels=True,title='Cone of 45°')

# I_D4_C45.plot_splash_traj(np.linspace(0,2,50),nolabels=True,title='Cone of 45°')

# # I_D5_C45.plot_splash_traj(np.linspace(0,3,3),nolabels=True,title='Cone of 45°')

# # I_D6_C45.plot_splash_traj(np.linspace(0,3,3),nolabels=True,title='Cone of 45°')




#### Drop volume fraction in jets

In [11]:
# fig,ax = plt.subplots(dpi =200)
# fig.suptitle('Drop of 3mm diameter')
# ax.set_xlabel('Off-centering/Cone radius')
# ax.set_ylabel('Drop fraction in the jet (%)')

# fig1,ax1 = plt.subplots(dpi =200)
# fig1.suptitle('Drop of 3mm diameter')
# ax1.set_xlabel('Off-centering/Cone radius')
# ax1.set_ylabel('Impact fraction in the jet (%)')


# # # Cones
# # C45_27 = dgc.Cone(2.7,np.pi/4)
# # C45_54 = dgc.Cone(5.4,np.pi/4)

# # OffCents_27 = np.linspace(0.5,5,10)
# # Drops_27 = [dgc.Drop(2.9,x,50,4) for x in OffCents_27]
# # Impacts_27 = [C45_27.impact(D) for D in Drops_27]
# # JetFracs_27 = [I.JetFrac for I in Impacts_27]
# # DropFracs_27 = [I.JetFrac*I.VolFrac/100 for I in Impacts_27]

# ax.plot(OffCents_27/2.7,DropFracs_27,'-*',label='Cone 2.7 mm')
# ax1.plot(OffCents_27/2.7,JetFracs_27,'-*',label='Cone 2.7 mm')

# # OffCents_54 = np.linspace(0.5,8,10)
# # Drops_54 = [dgc.Drop(2.9,x,50,4) for x in OffCents_54]
# # Impacts_54 = [C45_54.impact(D) for D in Drops_54]
# # DropFracs_54 = [I.JetFrac*I.VolFrac/100 for I in Impacts_54]
# # JetFracs_54 = [I.JetFrac for I in Impacts_54]

# ax.plot(OffCents_54/5.4,DropFracs_54,'-*',label='Cone 5.4 mm')
# ax1.plot(OffCents_54/5.4,JetFracs_54,'-*',label='Cone 5.4 mm')

# plt.legend()


In [12]:

# def PhaseDiagram(nPts,coneR,Alphas):
    
#     for Alpha in Alphas:
        
#         print('\nAlpha = ' + str(Alpha/(2*np.pi)*360) + '°')
        
        
#         Pts = np.linspace(0,nPts-1,nPts).astype(int)

#         dropX = np.linspace(0,3*coneR,nPts) # Drop center X (in cone configuration)
#         dropR = np.linspace(coneR/2,2*coneR,nPts) # Drop radius (cone config)

#         dropXgrid,dropRgrid = np.meshgrid(dropX,dropR)

#         splashType = np.empty((nPts,nPts))

#         # criterion for splash sorting based on fraction of rop volume impacting the cone, 
#         # impact volume fraction in the jet and geometrical parameters
#         def sortSplash(JVF,DF,coneR,dropR,dropX):

#             splashType = -1 # undefined

#             if JVF>=90: # 90% or more of impacting volume in the jet
#                 splashType = 1 # Jet

#             elif JVF>=70: # 70-95% of impacting volume in the jet
#                 splashType = 2 # Transition SJ

#             elif DF <= 50: # 70% or more of the impacting volume is in the splash
#                 # and maximum 50% of drop volume impacts the cone
#                 splashType = 3 # Bell

#             else:
#                 splashType = 4 # Crown

#             return splashType

#         for i in Pts:
#             for j in Pts:

#                 dx = dropX[i]
#                 dr = dropR[j]

#                 text = 'Computing for : ConeR = ' + str(coneR) + ', dropR = ' + str(dr) + ', Offcent = ' + str(dx)

#                 print(text.ljust(10), end= '\r')

#                 if ((dx/dr)-(coneR/dr))<1:

#                     DF = volFrac([dx],dr,coneR) # volFrac(r,Rd,Rc)
#                     JVF = JetVolFrac3D(coneR,Alpha,dr,[dx],100) # JetVolFrac3D(coneR,Alpha,dropR,Xdrops,nptmesh)

#                     splashType[j,i] = sortSplash(JVF,DF,coneR,dr,dx)

#                 else:
#                     splashType[j,i] = 0 # no impact


#         fig,ax = plt.subplots(dpi=150,figsize = (7,6)) 
#         ax.set_title('Alpha = ' + str(Alpha/(2*np.pi)*360) + '°')
#         sc = ax.scatter(dropXgrid/coneR,dropRgrid/coneR,c=splashType,cmap='jet')
#         ax.set_xlabel('Offcent/ConeRadius')
#         ax.set_ylabel('DropSize/ConeSize')
    
#         cbar = plt.colorbar(sc, ticks = [-1,0,1,2,3,4])
#         cbar.ax.set_yticklabels(['undefined','No impact', 'Jet', 'Transition', 'Bell sheet','Crown sheet']);  # vertically oriented colorbar
#         fig.tight_layout()
            

In [13]:
# import pandas as pd
# data = pd.read_csv(r'd:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\DataAna\all_angles.csv')

In [14]:
# data = data.loc[(data['angle[degrees]']==45) &( data['IdealDiam[mm]']==2.7)]
# data

In [15]:
# Rds = data['DropDiam'].values/2
# # Rcs = data['TargetDiam'].values
# OffCs = data['OffCentering'].values
# Names = data['Ind.1']

In [16]:
# Drops = [dgc.Drop(Rd,OffC,35,5) for Rd,OffC in zip(Rds,OffCs)]
# C45 = dgc.Cone(2.7/2,np.pi/4)
# Impacts = [C45.impact(D) for D in Drops]

In [17]:
# C45.impact(dgc.Drop(1.8,0.315,60,5)).plot_splash_traj(np.linspace(0,10,51),nolabels=True)

In [18]:
# for t,i in zip(np.linspace(0,10,51),np.linspace(0,50,51)):
#     f,a = C45.impact(dgc.Drop(1.8,0.315,300,5)).plot_splash_traj([t],nolabels=True)
    
#     f.savefig(r'd:\Users\laplaud\Desktop\PostDoc\Code\DropProject_WithAna\TestFilm2\\' + str(i) + '.png' )
#     plt.close(f)

In [19]:
# for I,name in zip(Impacts,Names):
#     I.plot_splash_traj(np.linspace(0,5,5),nolabels=False,title=name)

# Impacts[0].plot_splash_traj(np.linspace(0,4.5,15),nolabels=False)